# Klasifikasi Berita

## Preprocesing

In [2]:
pip install pandas scikit-learn nltk sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd

# Load data from berita
df = pd.read_csv('Berita.csv')

In [5]:
# Display basic information about the dataset
print("Dataset shape:", df.shape)
df

Dataset shape: (1500, 6)


,No,judul,berita,tanggal,kategori,link
0,1,Airlangga Harap Kenaikan UMP Tingkatkan Daya B...,Menteri Koordinator (Menko) Bidang Perekonomia...,"Minggu, 01 Des 2024 23:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
1,2,PT SIER Beri Penghargaan untuk 50 Tenant Terba...,"Dalam rangka memeriahkan hari jadi ke-50, PT S...","Minggu, 01 Des 2024 20:45 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412012...
2,3,Prabowo Bakal Bentuk Kementerian Penerimaan Ne...,Wacana Presiden Prabowo Subianto akan membentu...,"Minggu, 01 Des 2024 19:40 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
3,4,Sinergi Kemenag & BPJS Ketenagakerjaan Lindung...,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,"Minggu, 01 Des 2024 19:03 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
4,5,Pemerintah Segera Bentuk Satgas PHK Usai Tetap...,Pemerintah akan segera membentuk Satuan Tugas ...,"Minggu, 01 Des 2024 19:00 WIB",Ekonomi,https://www.cnnindonesia.com/ekonomi/202412011...
...,...,...,...,...,...,...
1495,1496,Laporan Sebab Tabrakan Pesawat-Black Hawk Dita...,Anggota Dewan Keselamatan Transportasi Nasiona...,"Jumat, 31 Jan 2025 04:40 WIB",Internasional,https://www.cnnindonesia.com/internasional/202...
1496,1497,"Israel Bebaskan 110 Sandera Palestina, Diantar...",Israel telah membebaskan 110 tahanan Palestina...,"Jumat, 31 Jan 2025 03:01 WIB",Internasional,https://www.cnnindonesia.com/internasional/202...
1497,1498,Hamas Konfirmasi Kematian Komandan Al Qassam M...,Hamas mengonfirmasi kematian kepala militernya...,"Jumat, 31 Jan 2025 02:30 WIB",Internasional,https://www.cnnindonesia.com/internasional/202...
1498,1499,Black Box American Airlines Ditemukan Usai Tab...,Tim penyelam diduga menemukan satu dari dua bl...,"Jumat, 31 Jan 2025 01:00 WIB",Internasional,https://www.cnnindonesia.com/internasional/202...


In [6]:
print("\nDataset info:")
df.info()


Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   No        1500 non-null   int64 
 1   judul     1500 non-null   object
 2   berita    1500 non-null   object
 3   tanggal   1500 non-null   object
 4   kategori  1500 non-null   object
 5   link      1500 non-null   object
dtypes: int64(1), object(5)
memory usage: 70.4+ KB


In [7]:
# Tampilkan data dari kolom "berita" dan "kategori"
df[['berita']]

,berita
0,Menteri Koordinator (Menko) Bidang Perekonomia...
1,"Dalam rangka memeriahkan hari jadi ke-50, PT S..."
2,Wacana Presiden Prabowo Subianto akan membentu...
3,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...
4,Pemerintah akan segera membentuk Satuan Tugas ...
...,...
1495,Anggota Dewan Keselamatan Transportasi Nasiona...
1496,Israel telah membebaskan 110 tahanan Palestina...
1497,Hamas mengonfirmasi kematian kepala militernya...
1498,Tim penyelam diduga menemukan satu dari dua bl...


In [8]:
# 1. Hapus baris jika kolom 'berita' ATAU 'kategori' kosong
# Kita butuh keduanya, jadi jika salah satu kosong, datanya tidak berguna.
df.dropna(subset=['berita'], inplace=True)

# 2. Hapus baris jika ada isi 'berita' yang sama persis (duplikat)
# Ini untuk memastikan setiap artikel unik dan model tidak dilatih dengan data yang berulang.
df.drop_duplicates(subset=['berita'], inplace=True)

In [9]:
import re

# Fungsi untuk membersihkan teks (menghapus tanda baca dan angka)
def clean_text(text):
    text = text.lower()  # Ubah ke huruf kecil
    text = re.sub(r'[^\w\s]', '', text)  # Hapus tanda baca
    text = re.sub(r'\d+', '', text)  # Hapus angka
    return text

# Terapkan fungsi pembersihan ke kolom 'berita' dan simpan di kolom baru 'berita_bersih'
df['berita_bersih'] = df['berita'].apply(clean_text)
# Tampilkan perbandingan kolom asli ('berita') dan kolom yang sudah dibersihkan ('berita_bersih')
print("Perbandingan Teks Asli dan Teks Bersih:")
print(df[['berita', 'berita_bersih']].head())

Perbandingan Teks Asli dan Teks Bersih:
                                              berita  \
0  Menteri Koordinator (Menko) Bidang Perekonomia...   
1  Dalam rangka memeriahkan hari jadi ke-50, PT S...   
2  Wacana Presiden Prabowo Subianto akan membentu...   
3  BPJS Ketenagakerjaan dan Kementerian Agama (Ke...   
4  Pemerintah akan segera membentuk Satuan Tugas ...   

                                       berita_bersih  
0  menteri koordinator menko bidang perekonomian ...  
1  dalam rangka memeriahkan hari jadi ke pt surab...  
2  wacana presiden prabowo subianto akan membentu...  
3  bpjs ketenagakerjaan dan kementerian agama kem...  
4  pemerintah akan segera membentuk satuan tugas ...  


In [10]:
import sys

# Perintah untuk menginstal library menggunakan path Python yang sedang aktif
!{sys.executable} -m pip install nltk

In [12]:
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK data (ensure these are downloaded)
nltk.download('punkt')
# nltk.download('punkt_tab') # This seems specific and might not be needed for basic tokenization

# Fungsi untuk melakukan tokenisasi
def tokenize_text(text):
    # Ensure the input is a string before tokenization
    if isinstance(text, str):
        return word_tokenize(text)
    else:
        return [] # Return an empty list or handle non-string as appropriate

# Terapkan tokenisasi ke kolom 'berita_bersih'
df['tokenized_berita'] = df['berita_bersih'].apply(tokenize_text)

# Tampilkan DataFrame with tokenized columns
display(df[['berita_bersih', 'tokenized_berita']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,berita_bersih,tokenized_berita
0,menteri koordinator menko bidang perekonomian ...,"[menteri, koordinator, menko, bidang, perekono..."
1,dalam rangka memeriahkan hari jadi ke pt surab...,"[dalam, rangka, memeriahkan, hari, jadi, ke, p..."
2,wacana presiden prabowo subianto akan membentu...,"[wacana, presiden, prabowo, subianto, akan, me..."
3,bpjs ketenagakerjaan dan kementerian agama kem...,"[bpjs, ketenagakerjaan, dan, kementerian, agam..."
4,pemerintah akan segera membentuk satuan tugas ...,"[pemerintah, akan, segera, membentuk, satuan, ..."


In [13]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Dapatkan Stop Word bahasa Indonesia
list_stopwords = set(stopwords.words('indonesian'))

# Fungsi untuk menghapus stop words
def remove_stopwords(tokens):
    return [word for word in tokens if word not in list_stopwords]

# Terapkan penghapusan Stop Word ke kolom 'tokenized_isi'
df['stopwords_removed_berita'] = df['tokenized_berita'].apply(remove_stopwords)
df['stopwords_removed_berita'] = df['tokenized_berita'].apply(remove_stopwords)

# Tampilkan DataFrame
display(df[['tokenized_berita', 'stopwords_removed_berita']].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tokenized_berita,stopwords_removed_berita
0,"[menteri, koordinator, menko, bidang, perekono...","[menteri, koordinator, menko, bidang, perekono..."
1,"[dalam, rangka, memeriahkan, hari, jadi, ke, p...","[rangka, memeriahkan, pt, surabaya, industrial..."
2,"[wacana, presiden, prabowo, subianto, akan, me...","[wacana, presiden, prabowo, subianto, membentu..."
3,"[bpjs, ketenagakerjaan, dan, kementerian, agam...","[bpjs, ketenagakerjaan, kementerian, agama, ke..."
4,"[pemerintah, akan, segera, membentuk, satuan, ...","[pemerintah, membentuk, satuan, tugas, pemutus..."


In [14]:
!{sys.executable} -m pip install Sastrawi

In [15]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from IPython.display import display
# Asumsikan 'df' adalah DataFrame Anda yang sudah ada

# Buat stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk melakukan stemming
def stem_tokens(tokens):
    return [stemmer.stem(token) for token in tokens]

# Terapkan stemming ke kolom 'stopwords_removed_berita'
df['stemmed_berita'] = df['stopwords_removed_berita'].apply(stem_tokens)

# Function to find and format stemmed word changes
def get_stemming_changes(original_tokens, stemmed_tokens):
    changes = []
    for original, stemmed in zip(original_tokens, stemmed_tokens):
        if original != stemmed:
            changes.append(f"{original} : {stemmed}")
    return changes

# Dapatkan semua perubahan stemming dari kolom yang benar
all_changes = []
for index, row in df.iterrows():
    # PERBAIKAN: Ganti 'stemmed_isi' menjadi 'stemmed_berita'
    all_changes.extend(get_stemming_changes(row['stopwords_removed_berita'], row['stemmed_berita']))

# Dapatkan perubahan unik dan urutkan
unique_changes = sorted(list(set(all_changes)))

# Cetak perubahan unik
print("Kata-kata yang mengalami stemming:")
for change in unique_changes:
    print(change)

# Tampilkan DataFrame
print("\nContoh hasil stemming:")
display(df[['stopwords_removed_berita', 'stemmed_berita']].head())

Kata-kata yang mengalami stemming:
abaikan : abai
absennya : absen
acuan : acu
adakah : ada
adangan : adang
adian : adi
adiknya : adik
adonan : adon
aduan : adu
advokasinya : advokasi
agendakan : agenda
agendanya : agenda
agensi : agens
airnya : air
ajakan : aja
ajaran : ajar
ajarannya : ajar
ajarkan : ajar
ajukan : aju
akademisnya : akademis
akibatnya : akibat
akrabnya : akrab
aksesi : akses
aksesnya : akses
aksinya : aksi
aktifkan : aktif
aktivitasnya : aktivitas
akui : aku
alasan : alas
alasannya : alas
alati : alat
alfan : alf
alihan : alih
alihkan : alih
aliran : alir
alokasikan : alokasi
alokasinya : alokasi
aluminanya : alumina
alunan : alun
amali : amal
amanatnya : amanat
amani : aman
amankan : aman
ambulan : ambul
amini : amin
ampunan : ampun
ampuni : ampun
amukan : amuk
anai : ana
anakku : anak
anaknya : anak
analisi : analis
ancaman : ancam
ancamannya : ancam
andalan : andal
andalannya : andal
aneksasinya : aneksasi
anggapan : anggap
anggaran : anggar
anggarannya : anggar
an

,stopwords_removed_berita,stemmed_berita
0,"[menteri, koordinator, menko, bidang, perekono...","[menteri, koordinator, menko, bidang, ekonomi,..."
1,"[rangka, memeriahkan, pt, surabaya, industrial...","[rangka, riah, pt, surabaya, industrial, estat..."
2,"[wacana, presiden, prabowo, subianto, membentu...","[wacana, presiden, prabowo, subianto, bentuk, ..."
3,"[bpjs, ketenagakerjaan, kementerian, agama, ke...","[bpjs, ketenagakerjaan, menteri, agama, kemena..."
4,"[pemerintah, membentuk, satuan, tugas, pemutus...","[perintah, bentuk, satu, tugas, putus, hubung,..."


In [16]:
from collections import Counter

# Gabungkan semua token dari 'stemmed_berita' menjadi satu daftar
# PERBAIKAN: Mengganti 'stemmed_isi' menjadi 'stemmed_berita'
all_stemmed_words = [word for tokens in df['stemmed_berita'] for word in tokens]

# Hitung frekuensi setiap kata
word_frequencies = Counter(all_stemmed_words)

# Menampilkan kata-kata yang paling umum dan frekuensinya
print("Top Most Frequent Words:")
# Gunakan .most_common() untuk mendapatkan urutan dari yang paling sering
for word, frequency in word_frequencies.most_common():
    print(f"{word}: {frequency}")

# Jika Anda ingin melihat 20 teratas saja, gunakan .most_common(20)
# print("\nTop 20 Most Frequent Words:")
# for word, frequency in word_frequencies.most_common(20):
#     print(f"{word}: {frequency}")

Top Most Frequent Words:
indonesia: 2325
menteri: 1355
laut: 1311
perintah: 1267
cnn: 1249
main: 1244
presiden: 1113
rp: 1021
orang: 956
jalan: 955
negara: 949
pagar: 918
israel: 907
timnas: 891
persen: 885
milik: 876
hasil: 813
jakarta: 809
piala: 753
tim: 718
tahan: 713
warga: 703
kerja: 699
laku: 698
latih: 695
prabowo: 690
pilih: 690
gambasvideo: 670
temu: 646
duga: 645
masyarakat: 619
senjata: 613
terima: 611
hukum: 607
menang: 604
lapor: 598
gol: 594
menit: 583
kait: 581
rumah: 580
bakar: 578
program: 574
korban: 570
usaha: 567
serang: 567
wilayah: 562
bangun: 556
salah: 544
gencat: 535
uang: 515
gaza: 507
laga: 503
putus: 502
makan: 497
tingkat: 494
dukung: 494
langsung: 488
as: 488
dunia: 486
tanding: 476
aku: 467
sebut: 461
capai: 460
liga: 459
jabat: 458
harga: 457
anggota: 455
kota: 453
nasional: 452
lawan: 452
bantu: 450
juta: 445
korea: 445
poin: 444
minggu: 439
resmi: 433
selatan: 425
januari: 425
aman: 424
air: 422
trump: 418
gizi: 414
kali: 407
aff: 406
senin: 404
ri: 4

In [18]:
import pandas as pd # Pastikan pandas sudah di-import

# Buat DataFrame baru dengan isi asli, isi stemmed, dan kategori
# PERBAIKAN: Ganti 'stemmed_isi' menjadi 'stemmed_berita'
# Asumsi: Anda memiliki kolom 'isi_berita' dan 'kategori' di df asli Anda.
processed_df = df[['berita', 'stemmed_berita', 'kategori']].copy()

# Ganti nama kolom
# PERBAIKAN: Ganti key di dictionary dari 'stemmed_isi' menjadi 'stemmed_berita'
processed_df.rename(columns={'stemmed_berita': 'hasil_preprocessing'}, inplace=True)

# Konversi frekuensi kata ke DataFrame
# (Pastikan variabel 'word_frequencies' sudah ada dari kode sebelumnya)
frequency_df = pd.DataFrame.from_dict(word_frequencies, orient='index', columns=['frequency'])
frequency_df.index.name = 'word'
frequency_df.sort_values(by='frequency', ascending=False, inplace=True)

# Simpan ke dua file CSV terpisah
processed_df.to_csv('hasil_preprocessing_berita.csv', index=False, encoding='utf-8')
frequency_df.to_csv('frek_kata_berita.csv', encoding='utf-8')

print("Hasil preprocessing disimpan di 'hasil_preprocessing_berita.csv'")
print("Frekuensi kata disimpan di 'frek_kata_berita.csv'")

Hasil preprocessing disimpan di 'hasil_preprocessing_berita.csv'
Frekuensi kata disimpan di 'frek_kata_berita.csv'


In [19]:
hasil_preprocessing = "hasil_preprocessing_berita.csv"
df = pd.read_csv(hasil_preprocessing)

# Tampilkan data
df

,berita,hasil_preprocessing,kategori
0,Menteri Koordinator (Menko) Bidang Perekonomia...,"['menteri', 'koordinator', 'menko', 'bidang', ...",Ekonomi
1,"Dalam rangka memeriahkan hari jadi ke-50, PT S...","['rangka', 'riah', 'pt', 'surabaya', 'industri...",Ekonomi
2,Wacana Presiden Prabowo Subianto akan membentu...,"['wacana', 'presiden', 'prabowo', 'subianto', ...",Ekonomi
3,BPJS Ketenagakerjaan dan Kementerian Agama (Ke...,"['bpjs', 'ketenagakerjaan', 'menteri', 'agama'...",Ekonomi
4,Pemerintah akan segera membentuk Satuan Tugas ...,"['perintah', 'bentuk', 'satu', 'tugas', 'putus...",Ekonomi
...,...,...,...
1495,Anggota Dewan Keselamatan Transportasi Nasiona...,"['anggota', 'dewan', 'selamat', 'transportasi'...",Internasional
1496,Israel telah membebaskan 110 tahanan Palestina...,"['israel', 'bebas', 'tahan', 'palestina', 'kam...",Internasional
1497,Hamas mengonfirmasi kematian kepala militernya...,"['hamas', 'konfirmasi', 'mati', 'kepala', 'mil...",Internasional
1498,Tim penyelam diduga menemukan satu dari dua bl...,"['tim', 'selam', 'duga', 'temu', 'black', 'box...",Internasional


### modelling mengunakan classifier naïve bayes dan SVM 

In [20]:
import pandas as pd
import ast  # Untuk mengubah string-list menjadi list
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Tentukan jumlah topik yang ingin diekstraksi
# Ini adalah hyperparameter, bisa diubah (misal: 10, 20, 50)
N_TOPICS = 20

# --- 1. Persiapan Data (Load dan Cleaning) ---

# Muat data
try:
    df = pd.read_csv('hasil_preprocessing_berita.csv')
except Exception as e:
    print(f"Error saat membaca file: {e}")
    # Jika error, buat dataframe dummy untuk melanjutkan (ganti dengan file Anda)
    # df = pd.DataFrame({
    #     'hasil_preprocessing': ["['satu', 'dua', 'tiga']", "['empat', 'lima', 'enam']", "['satu', 'lima', 'tujuh']"],
    #     'kategori': ['A', 'B', 'A']
    # })

print("Data berhasil dimuat.")
print(f"Jumlah data: {len(df)} baris")

# Cleaning kolom 'hasil_preprocessing'
# 1. Isi data kosong (NaN) dengan string list kosong '[]'
df['hasil_preprocessing'] = df['hasil_preprocessing'].fillna('[]')

# 2. Ubah string-list menjadi list sungguhan
#    Contoh: "['kata', 'satu']" -> ['kata', 'satu']
try:
    df['tokens'] = df['hasil_preprocessing'].apply(ast.literal_eval)
except ValueError as e:
    print(f"Error saat konversi string ke list: {e}")
    print("Pastikan format di CSV Anda adalah '[\"kata1\", \"kata2\"]' atau '[\'kata1\', \'kata2\']'")
    # Jika gagal, coba cara yang lebih 'memaksa' (tidak disarankan jika data kotor)
    # df['tokens'] = df['hasil_preprocessing'].apply(lambda x: x.strip("[]").replace("'", "").replace("\"", "").split(', '))


# 3. Gabungkan kembali token menjadi satu string/kalimat
#    Contoh: ['kata', 'satu'] -> "kata satu"
df['text_joined'] = df['tokens'].apply(lambda x: ' '.join(x))

print("Data cleaning selesai. Kolom 'text_joined' siap untuk vektorisasi.")
# print(df[['hasil_preprocessing', 'tokens', 'text_joined']].head())


# --- 2. Enkode Label (Target) ---

le = LabelEncoder()
df['kategori_encoded'] = le.fit_transform(df['kategori'])
y = df['kategori_encoded']

# Simpan nama kategori untuk laporan nanti
kategori_names = le.classes_
print(f"Kategori ditemukan: {kategori_names}")


# --- 3. Ekstraksi Fitur (Topic Modeling dengan TF-IDF + NMF) ---

print("Memulai ekstraksi fitur dengan TF-IDF...")
# Buat matriks TF-IDF
# max_df=0.95 -> abaikan kata yang terlalu sering (muncul di > 95% dokumen)
# min_df=2 -> abaikan kata yang terlalu jarang (muncul di < 2 dokumen)
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_joined'])

print(f"Bentuk matriks TF-IDF: {tfidf_matrix.shape}")

print(f"Memulai Topic Modeling (NMF) dengan {N_TOPICS} topik...")
# Terapkan NMF untuk mereduksi dimensi matriks TF-IDF
# menjadi matriks fitur topik
nmf = NMF(n_components=N_TOPICS, random_state=42, init='nndsvda', l1_ratio=0)
# 'X' sekarang adalah matriks fitur kita (Dokumen x Topik)
X = nmf.fit_transform(tfidf_matrix)

print(f"Bentuk matriks fitur (topik): {X.shape}")


# --- 4. Persiapan Training (Split Data) ---

# Bagi data menjadi 80% data latih dan 20% data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("\nData berhasil dibagi menjadi data latih dan data uji.")
print(f"X_train shape: {X_train.shape} | y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}  | y_test shape: {y_test.shape}")


# --- 5. Model 1: Klasifikasi dengan Gaussian Naïve Bayes ---

print("\n--- Melatih Model Gaussian Naïve Bayes ---")
# Kita menggunakan GaussianNB karena fitur hasil NMF adalah nilai kontinu (bukan hitungan/count)
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Prediksi data uji
y_pred_gnb = gnb.predict(X_test)

# Tampilkan hasil
print("Hasil Klasifikasi dengan Naïve Bayes:")
print(f"Akurasi: {accuracy_score(y_test, y_pred_gnb):.4f}")
print(classification_report(y_test, y_pred_gnb, target_names=kategori_names))


# --- 6. Model 2: Klasifikasi dengan Support Vector Machine (SVM) ---

print("\n--- Melatih Model Support Vector Machine (SVM) ---")
# Menggunakan kernel 'linear' seringkali jadi baseline yang kuat
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)

# Prediksi data uji
y_pred_svm = svm.predict(X_test)

# Tampilkan hasil
print("Hasil Klasifikasi dengan SVM:")
print(f"Akurasi: {accuracy_score(y_test, y_pred_svm):.4f}")
print(classification_report(y_test, y_pred_svm, target_names=kategori_names))

Data berhasil dimuat.
Jumlah data: 1500 baris
Data cleaning selesai. Kolom 'text_joined' siap untuk vektorisasi.
Kategori ditemukan: ['Ekonomi' 'Internasional' 'Nasional' 'Olahraga']
Memulai ekstraksi fitur dengan TF-IDF...
Bentuk matriks TF-IDF: (1500, 10350)
Memulai Topic Modeling (NMF) dengan 20 topik...
Bentuk matriks fitur (topik): (1500, 20)

Data berhasil dibagi menjadi data latih dan data uji.
X_train shape: (1200, 20) | y_train shape: (1200,)
X_test shape: (300, 20)  | y_test shape: (300,)

--- Melatih Model Gaussian Naïve Bayes ---
Hasil Klasifikasi dengan Naïve Bayes:
Akurasi: 0.8733
               precision    recall  f1-score   support

      Ekonomi       0.82      0.83      0.82        75
Internasional       0.88      0.95      0.91        75
     Nasional       0.82      0.73      0.77        75
     Olahraga       0.97      0.99      0.98        75

     accuracy                           0.87       300
    macro avg       0.87      0.87      0.87       300
 weighted a